In [216]:
import numpy as np
import os
import plotly.graph_objects as go
from sklearn.manifold import TSNE

# モデルのパスやパラメータの設定
model_path = '../output_WN18RR/embedding'
embedding_range = 0.01
model = 'kg-bert'
entities_dict_text_file = '../data/dict/WN18RR_text/entities.dict'
entities_dict_number_file = '../data/dict/WN18RR/entities.dict'
train_file = '../data/dict/WN18RR/train.txt'
target_values = [ ["flower", 'red'], ["man", 'green'],["plant genus", 'blue'], ["country", 'orange'], ["sport", 'magenta'], ["music", 'grey'], ["bird", 'pink']]
# target_values = [["mammal genus", 'brown'],["genus.n.02", 'red'], ["dog.n.01", 'blue'], ["clothing.n.01", 'orange'], ["plant_genus.n.01", 'green'],["mammal_genus.n.01", 'yellow'], ["bird_genus.n.01", 'purple'], ["arthropod.n.01", 'gray']]



fig = go.Figure()

# エンティティの埋め込みをロード
print("エンティティ埋め込みをロード中...")
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))
print(f"エンティティ埋め込みの形状: {entity_embedding.shape}")

def load_entities_dict(file_path):
    entities_dict = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            entities_dict[int(parts[0])] = parts[1]
    return entities_dict

# エンティティ名の辞書をロード
entities_dict_text = load_entities_dict(entities_dict_text_file)
entities_dict_number = load_entities_dict(entities_dict_number_file)
print(f"エンティティ名の辞書（entities_dict_text）が {len(entities_dict_text)} エントリーでロードされました")
print(f"エンティティ名の辞書（entities_dict_number）が {len(entities_dict_number)} エントリーでロードされました")


エンティティ埋め込みをロード中...
エンティティ埋め込みの形状: (40943, 768)
エンティティ名の辞書（entities_dict_text）が 40943 エントリーでロードされました
エンティティ名の辞書（entities_dict_number）が 40943 エントリーでロードされました


In [217]:
def extract_and_search_data(target_value, train_file_path, entities_dict_text, entities_dict_number):
    # エンティティ名のtext辞書読み込み
    def load_entities_dict(file_path):
        entities_dict = {}
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                entities_dict[int(parts[0])] = parts[1]
        return entities_dict

    entities_dict_text = load_entities_dict(entities_dict_text_file)
    entities_dict_number = load_entities_dict(entities_dict_number_file)

    # エンティティ名からインデックスを取得する関数
    def text_to_num(target_value):
        for index, value in entities_dict_text.items():
            if value == target_value:
                return entities_dict_number[index]

    # 対象の値からインデックスを取得
    num = text_to_num(target_value)

    # ファイルからデータを抽出する関数
    def extract_data(file_path, target_value):
        target_value = "\t" + target_value
        extracted_data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if target_value in line:
                    parts = line.strip().split('\t')
                    extracted_data.append(parts[0])
        return extracted_data

    # ファイルからデータを検索する関数
    def search_num(extracted_data, file_path):
        data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                for i in extracted_data:
                    if i in line:
                        parts = line.strip().split('\t')
                        data.append(parts[0])
        return data

    # データを抽出
    extracted_data = extract_data(train_file_path, num)

    # データを検索
    searched_data = search_num(extracted_data, entities_dict_number_file)

    return searched_data

# 全データを格納するリスト
all_extracted_data = []
all_target_colors = []

# 対象の値と色を指定してデータをロードし、リストに追加するループ
for target_value, target_color in target_values:
    print(f"Processing target_value: {target_value}, target_color: {target_color}")
    extracted_data = extract_and_search_data(target_value, train_file, entities_dict_text, entities_dict_number)
    print(f"{target_value}: {extracted_data}")
    all_extracted_data.extend(extracted_data)
    all_target_colors.extend([target_color] * len(extracted_data))


print(f"Final all_extracted_data: {all_extracted_data}")
print(f"Final all_target_colors: {all_target_colors}")


Processing target_value: flower, target_color: red
flower: ['476', '793', '1563', '1677', '2274', '2647', '4072', '6638', '9581', '11230', '12491', '13213', '15625', '16243', '16397', '17416', '19493', '20665', '21774', '24410', '24730', '25536', '26677', '28016', '28231', '29210', '30423', '30903', '31029', '32057', '33009', '33022', '33039', '33153', '33217', '33424', '33439', '33491', '33672', '33701', '33711', '33719', '35003', '40368', '40450']
Processing target_value: man, target_color: green
man: ['736', '975', '13009', '15201', '15512', '15965', '18528', '19719', '21194', '22465', '24428', '25521', '34099', '34184', '35956', '36653', '37732']
Processing target_value: plant genus, target_color: blue
plant genus: ['493', '2074', '2867', '5662', '5867', '6899', '7119', '9326', '10637', '11107', '13568', '13667', '15472', '16624', '17334', '18091', '19357', '19819', '19917', '23660', '25702', '26665', '27206', '28945', '30957', '30958', '30959', '31058', '31062', '31072', '31672', 

In [218]:
print(entity_embedding[1])


[ 3.88207257e-01 -9.72070992e-02 -9.45599526e-02  8.58605802e-01
 -2.51535401e-02 -1.90329235e-02  1.28654301e-01 -2.28415892e-01
  6.74083382e-02 -2.32587397e-01  6.79172352e-02 -7.60704204e-02
  1.26143754e-01  1.16332863e-02  7.21743330e-02  6.27127364e-02
 -3.27853113e-02 -3.33873898e-01  2.11903965e-03 -4.80807364e-01
  1.24034040e-01  3.39350034e-03  6.50489405e-02 -1.97913125e-01
  1.08516820e-01 -1.73814997e-01  8.27156454e-02  3.97313982e-01
  2.26008371e-01  7.90359020e-01 -7.43637839e-03  1.56331539e-01
  2.62306482e-02 -1.03906356e-01  1.25504527e-02 -2.42884997e-02
 -7.31201842e-02 -2.26540230e-02 -1.96807280e-01  2.00937048e-01
 -3.25276405e-01 -6.25790134e-02  8.72456193e-01 -2.54155099e-01
  8.33766386e-02 -4.72271234e-01  2.26590082e-01  6.98451579e-01
 -1.23654502e-02 -1.34362221e-01  9.43811536e-02  7.29963422e-01
  4.42411691e-01 -6.29173934e-01  1.80484220e-01 -1.91789836e-01
  1.95765853e-01 -3.10595155e-01  1.48961216e-01  1.02975434e-02
 -7.78486356e-02 -9.04358

In [221]:

def calculate_tsne_coordinates(extracted_data, entity_embedding, perplexity=5):
    # Get embeddings for extracted data
    data_embeddings = np.array([entity_embedding[int(entity_id_str)] for entity_id_str in extracted_data])
    
    # Apply t-SNE to reduce dimensionality to 2D
    tsne = TSNE(n_components=2, random_state=0, perplexity=perplexity)
    tsne_coordinates = tsne.fit_transform(data_embeddings)
    
    return tsne_coordinates.tolist()

# t-SNE座標を計算
tsne_coordinates = calculate_tsne_coordinates(all_extracted_data, entity_embedding)

# プロットにトレースを追加するループ
for (x, y), color, entity_id_str in zip(tsne_coordinates, all_target_colors, all_extracted_data):
    entity_id = int(entity_id_str)
    entity_name = entities_dict_text[entity_id]
    fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', marker=dict(color=color, size=5), opacity=0.8, name=entity_name))

# 色と対応するエンティティの説明を右上に追加
for i, (target_value, target_color) in enumerate(target_values):
    fig.add_annotation(
        text=target_value,
        x=1.05,  # テキストのx座標 (図の右端から少し右)
        y=1 - i * 0.05,  # テキストのy座標 (図の上端から少し下)
        xref="paper",  # x座標の参照元 (paperは図の左端、dataはデータ座標)
        yref="paper",  # y座標の参照元 (paperは図の下端、dataはデータ座標)
        xanchor="left",  # テキストを左詰にする
        showarrow=False,  # 矢印を非表示にする
        font=dict(size=12, color=target_color),  # テキストのフォントサイズと色
    )


# HAKEテキストの追加
fig.add_annotation(
    text=model+' Entity Embedding',  # 表示するテキスト
    xref="paper",  # x座標の参照元 (paperは図の左端、dataはデータ座標)
    yref="paper",  # y座標の参照元 (paperは図の下端、dataはデータ座標)
    x=0.5,  # x座標 (0.5は図の中央)
    y=1.1,  # y座標 (1.05は図の上端から5%上)
    showarrow=False,  # 矢印を非表示にする
    font=dict(size=20),  # テキストのフォントサイズ
)

# レイアウトの設定
fig.update_layout(
    title="",
    yaxis_title=" ",
    showlegend=False,
    width=550,  # 幅
    height=500,  # 高さ
    xaxis=dict(scaleanchor="y", scaleratio=1),  # x軸のアスペクト比を1:1に設定
    yaxis=dict(scaleanchor="x", scaleratio=1),  # y軸のアスペクト比を1:1に設定
)

fig.show()


In [220]:
# モデルのパスやパラメータの設定
model_path = '../output_WN18RR/embedding'

fig = go.Figure()

# エンティティの埋め込みをロード
print("エンティティ埋め込みをロード中...")
relation_embedding = np.load(os.path.join(model_path, 'relation_embedding.npy'))

print(relation_embedding[0][0])
print(relation_embedding[1][0])
print(relation_embedding[2][0])



エンティティ埋め込みをロード中...
0.4229233
0.5278447
0.5169488
